<a href="https://colab.research.google.com/github/forallx94/log_anomaly_detection/blob/main/LDA-for-HDFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1차

In [1]:
!pip install pyldavis

     |████████████████████████████████| 1.6MB 6.0MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=73743294d8ad59f588b84c07d8985b5a8da847a58224e1b77daa127896a7b6e8
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyldavis


In [2]:
import re
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from pprint import pprint

In [92]:
df = pd.read_csv('./data/HDFS_100k.log_structured.csv')

In [4]:
df

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,E22,BLOCK* NameSystem.allocateBlock:<*>
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>
3,4,81109,203519,145,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>
4,5,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...,E11,PacketResponder <*> for block <*> terminating
...,...,...,...,...,...,...,...,...,...
104810,104811,81109,210122,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_-87387097785867...,E2,Verification succeeded for <*>
104811,104812,81109,210124,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_642047611142564...,E2,Verification succeeded for <*>
104812,104813,81109,210126,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_-23823897510323...,E2,Verification succeeded for <*>
104813,104814,81109,210126,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_485603173001003...,E2,Verification succeeded for <*>


In [ ]:
import nltk
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)
text.head()

In [5]:
data = [i.strip().split(' ') for i in df.Content]

In [6]:
id2word = corpora.Dictionary(data)

In [7]:
texts = data

In [8]:
corpus = [id2word.doc2bow(text) for text in texts]

In [9]:
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]]


In [10]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                            id2word = id2word,
                                            num_topics = 10,
                                            random_state = 100,
                                            update_every =1,
                                            chunksize = 100,
                                            passes = 10,
                                            alpha= 'auto',
                                            per_word_topics = True
                                            )

In [11]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.046*"BLOCK*" + 0.031*"NameSystem.allocateBlock:" + '
  '0.012*"blk_4241467193520768333" + 0.004*"blk_8909107483987085802" + '
  '0.004*"blk_9047244576407600764" + 0.004*"blk_524408193881860019" + '
  '0.004*"blk_4222802578485615752" + 0.004*"blk_-2951262078924395064" + '
  '0.004*"blk_-9081208173975932165" + 0.004*"blk_8652779608730262978"'),
 (1,
  '0.101*"to" + 0.099*"NameSystem.addStoredBlock:" + 0.099*"size" + 0.099*"is" '
  '+ 0.099*"blockMap" + 0.099*"added" + 0.099*"updated:" + 0.097*"67108864" + '
  '0.095*"BLOCK*" + 0.001*"10.251.39.160:50010"'),
 (2,
  '0.018*"blk_-2607603250149292767" + 0.017*"blk_4286887770593516283" + '
  '0.017*"blk_1037066694877136451" + 0.016*"10.251.74.79:50010" + '
  '0.016*"blk_-7390710706320943439" + 0.016*"blk_-7385019746290967026" + '
  '0.016*"blk_-1160765867676592212" + 0.015*"10.251.31.5:50010" + '
  '0.015*"blk_-4130422002907235753" + 0.015*"blk_-372632911736985200"'),
 (3,
  '0.157*"block" + 0.151*"of" + 0.150*"from" + 0.150*"size" 

In [12]:
import pyLDAvis
import pyLDAvis.gensim as gensimvis

In [13]:
# # 시각화에 1시간 이상 소모
# pyLDAvis.enable_notebook()
# vis = gensimvis.prepare(lda_model, corpus, id2word)
# pyLDAvis.display(vis)

KeyboardInterrupt: ignored

## 2차

* https://omicro03.medium.com/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC-nlp-8%EC%9D%BC%EC%B0%A8-lda-f571b4da9d04

In [40]:
text = df[['LineId','Content']]
text.head()

,LineId,Content
0,1,Receiving block blk_-1608999687919862906 src: ...
1,2,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...
2,3,Receiving block blk_-1608999687919862906 src: ...
3,4,Receiving block blk_-1608999687919862906 src: ...
4,5,PacketResponder 1 for block blk_-1608999687919...


In [41]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [42]:
# 토큰화
text['Content'] = text.apply(lambda row: nltk.word_tokenize(row['Content']), axis=1)
text.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,LineId,Content
0,1,"[Receiving, block, blk_-1608999687919862906, s..."
1,2,"[BLOCK*, NameSystem.allocateBlock, :, /mnt/had..."
2,3,"[Receiving, block, blk_-1608999687919862906, s..."
3,4,"[Receiving, block, blk_-1608999687919862906, s..."
4,5,"[PacketResponder, 1, for, block, blk_-16089996..."


In [43]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
# 불용어 처리
stop = stopwords.words('english')
text['Content'] = text['Content'].apply(lambda x: [word for word in x if word not in (stop)])
text.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,LineId,Content
0,1,"[Receiving, block, blk_-1608999687919862906, s..."
1,2,"[BLOCK*, NameSystem.allocateBlock, :, /mnt/had..."
2,3,"[Receiving, block, blk_-1608999687919862906, s..."
3,4,"[Receiving, block, blk_-1608999687919862906, s..."
4,5,"[PacketResponder, 1, block, blk_-1608999687919..."


In [45]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [46]:
# 표제어 추출
from nltk.stem import WordNetLemmatizer
text['Content'] = text['Content'].apply(lambda x : [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
text.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,LineId,Content
0,1,"[Receiving, block, blk_-1608999687919862906, s..."
1,2,"[BLOCK*, NameSystem.allocateBlock, :, /mnt/had..."
2,3,"[Receiving, block, blk_-1608999687919862906, s..."
3,4,"[Receiving, block, blk_-1608999687919862906, s..."
4,5,"[PacketResponder, 1, block, blk_-1608999687919..."


In [47]:
# 일정 길이 이하 단어 삭제 
tokenized_doc = text['Content'].apply(lambda x : [word for word in x if len(word) > 3])
tokenized_doc[:5]

0    [Receiving, block, blk_-1608999687919862906, /...
1    [BLOCK*, NameSystem.allocateBlock, /mnt/hadoop...
2    [Receiving, block, blk_-1608999687919862906, /...
3    [Receiving, block, blk_-1608999687919862906, /...
4    [PacketResponder, block, blk_-1608999687919862...
Name: Content, dtype: object

In [32]:
# 역토큰화
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
text['Content'] = detokenized_doc
# 다시 text['Content'] 에 저장
text.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,LineId,Content
0,1,Receiving block blk_-1608999687919862906 /10.2...
1,2,BLOCK* NameSystem.allocateBlock /mnt/hadoop/ma...
2,3,Receiving block blk_-1608999687919862906 /10.2...
3,4,Receiving block blk_-1608999687919862906 /10.2...
4,5,PacketResponder block blk_-1608999687919862906...


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(text['Content'])
X.shape

(104815, 1000)

In [34]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)
lda_top = lda_model.fit_transform(X)
print(lda_model.components_)
print(lda_model.components_.shape)

[[1.00000006e-01 1.00000006e-01 1.00000008e-01 ... 1.00000042e-01
  1.00000818e-01 1.00000007e-01]
 [1.89001672e+00 4.28374804e+00 2.82443277e+00 ... 2.00300039e+03
  1.00000107e-01 1.00000008e-01]
 [1.00000006e-01 1.00000009e-01 1.00000007e-01 ... 1.00000007e-01
  1.00000014e-01 1.00000007e-01]
 ...
 [1.00000008e-01 1.00000007e-01 1.00000007e-01 ... 1.00000008e-01
  1.00000403e-01 1.00000007e-01]
 [1.00017155e-01 1.00000006e-01 1.00000008e-01 ... 1.00000032e-01
  6.04580620e+03 1.00000007e-01]
 [1.00000006e-01 1.00000008e-01 1.00000007e-01 ... 1.00000122e-01
  1.00000184e-01 1.00000007e-01]]
(10, 1000)


In [38]:
terms = vectorizer.get_feature_names() 
# 단어 집합. 1,000개의 단어가 저장되어있음.
def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d :" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n -1:-1]])
get_topics(lda_model.components_, terms)

Topic 1 : [('107', 1102.18), ('251', 1081.04), ('30', 1007.07), ('10', 951.24), ('134', 779.46)]
Topic 2 : [('root', 2003.0), ('user', 2003.0), ('rand', 2003.0), ('allocateblock', 2002.79), ('_temporary', 2002.77)]
Topic 3 : [('addstoredblock', 6126.61), ('blockmap', 6124.14), ('update', 6124.14), ('namesystem', 5393.07), ('50010', 3903.47)]
Topic 4 : [('10', 1478.77), ('250', 1312.99), ('received', 1171.09), ('size', 1045.39), ('67108864', 969.62)]
Topic 5 : [('251', 1343.46), ('10', 1232.41), ('203', 1066.3), ('received', 1034.8), ('123', 994.32)]
Topic 6 : [('70', 1141.97), ('194', 1058.49), ('251', 869.47), ('67', 792.84), ('10', 749.25)]
Topic 7 : [('214', 1701.53), ('251', 1019.8), ('10', 978.46), ('received', 834.1), ('74', 778.95)]
Topic 8 : [('251', 1207.42), ('10', 1055.27), ('31', 1029.24), ('111', 1027.74), ('192', 923.02)]
Topic 9 : [('packetresponder', 15604.81), ('terminate', 15602.6), ('block', 6559.31), ('blk_', 6069.46), ('verification', 6045.81)]
Topic 10 : [('10', 1

## 3차
1차 + 2차

In [93]:
text = df[['LineId','Content']]
text.head()

,LineId,Content
0,1,Receiving block blk_-1608999687919862906 src: ...
1,2,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...
2,3,Receiving block blk_-1608999687919862906 src: ...
3,4,Receiving block blk_-1608999687919862906 src: ...
4,5,PacketResponder 1 for block blk_-1608999687919...


In [94]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [95]:
# 토큰화
text['Content'] = text.apply(lambda row: nltk.word_tokenize(row['Content']), axis=1)
text.head()

,LineId,Content
0,1,"[Receiving, block, blk_-1608999687919862906, s..."
1,2,"[BLOCK*, NameSystem.allocateBlock, :, /mnt/had..."
2,3,"[Receiving, block, blk_-1608999687919862906, s..."
3,4,"[Receiving, block, blk_-1608999687919862906, s..."
4,5,"[PacketResponder, 1, for, block, blk_-16089996..."


In [96]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [97]:
# 불용어 처리
stop = stopwords.words('english')
text['Content'] = text['Content'].apply(lambda x: [word for word in x if word not in (stop)])
text.head()

,LineId,Content
0,1,"[Receiving, block, blk_-1608999687919862906, s..."
1,2,"[BLOCK*, NameSystem.allocateBlock, :, /mnt/had..."
2,3,"[Receiving, block, blk_-1608999687919862906, s..."
3,4,"[Receiving, block, blk_-1608999687919862906, s..."
4,5,"[PacketResponder, 1, block, blk_-1608999687919..."


In [98]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [99]:
# 표제어 추출
from nltk.stem import WordNetLemmatizer
text['Content'] = text['Content'].apply(lambda x : [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
text.head()

,LineId,Content
0,1,"[Receiving, block, blk_-1608999687919862906, s..."
1,2,"[BLOCK*, NameSystem.allocateBlock, :, /mnt/had..."
2,3,"[Receiving, block, blk_-1608999687919862906, s..."
3,4,"[Receiving, block, blk_-1608999687919862906, s..."
4,5,"[PacketResponder, 1, block, blk_-1608999687919..."


In [100]:
# 일정 길이 이하 단어 삭제 
tokenized_doc = text['Content'].apply(lambda x : [word for word in x if len(word) > 3])
tokenized_doc[:5]

0    [Receiving, block, blk_-1608999687919862906, /...
1    [BLOCK*, NameSystem.allocateBlock, /mnt/hadoop...
2    [Receiving, block, blk_-1608999687919862906, /...
3    [Receiving, block, blk_-1608999687919862906, /...
4    [PacketResponder, block, blk_-1608999687919862...
Name: Content, dtype: object

In [104]:
data = [i for i in tokenized_doc]

In [105]:
id2word = corpora.Dictionary(data)

In [106]:
corpus = [id2word.doc2bow(text) for text in data]

In [107]:
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [108]:
import warnings 
warnings.filterwarnings('ignore')

In [109]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                            id2word = id2word,
                                            num_topics = 10,
                                            random_state = 100,
                                            update_every =1,
                                            chunksize = 100,
                                            passes = 10,
                                            alpha= 'auto',
                                            per_word_topics = True
                                            )

In [110]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.307*"block" + 0.015*"blk_-3059453549958878582" + 0.013*"/10.251.31.5" + '
  '0.012*"/10.251.126.5" + 0.011*"/10.250.7.32" + 0.011*"/10.251.203.129" + '
  '0.011*"/10.250.11.85" + 0.010*"/10.251.71.97" + '
  '0.010*"blk_-836233151529212612" + 0.009*"/10.251.39.192"'),
 (1,
  '0.234*"block" + 0.016*"blk_-2607603250149292767" + '
  '0.014*"blk_3126100339057783399" + 0.014*"10.251.74.79:50010" + '
  '0.014*"blk_-598955920782630733" + 0.013*"blk_4718466536212741328" + '
  '0.012*"/10.251.111.209" + 0.012*"10.251.111.209:50010" + '
  '0.011*"/10.251.111.37" + 0.011*"blk_7907797724760003565"'),
 (2,
  '0.200*"size" + 0.195*"67108864" + 0.101*"Received" + '
  '0.100*"NameSystem.addStoredBlock" + 0.100*"update" + 0.100*"blockMap" + '
  '0.091*"BLOCK*" + 0.001*"10.251.39.160:50010" + 0.001*"10.251.39.209:50010" '
  '+ 0.001*"10.251.107.227:50010"'),
 (3,
  '0.060*"BLOCK*" + 0.028*"NameSystem.allocateBlock" + '
  '0.004*"blk_8909107483987085802" + 0.004*"blk_9047244576407600764" + '
  '

## 1차 3차 단어 정리

1차 topic

In [81]:
_1st = [(0,
  '0.046*"BLOCK*" + 0.031*"NameSystem.allocateBlock:" + '
  '0.012*"blk_4241467193520768333" + 0.004*"blk_8909107483987085802" + '
  '0.004*"blk_9047244576407600764" + 0.004*"blk_524408193881860019" + '
  '0.004*"blk_4222802578485615752" + 0.004*"blk_-2951262078924395064" + '
  '0.004*"blk_-9081208173975932165" + 0.004*"blk_8652779608730262978"'),
 (1,
  '0.101*"to" + 0.099*"NameSystem.addStoredBlock:" + 0.099*"size" + 0.099*"is" '
  '+ 0.099*"blockMap" + 0.099*"added" + 0.099*"updated:" + 0.097*"67108864" + '
  '0.095*"BLOCK*" + 0.001*"10.251.39.160:50010"'),
 (2,
  '0.018*"blk_-2607603250149292767" + 0.017*"blk_4286887770593516283" + '
  '0.017*"blk_1037066694877136451" + 0.016*"10.251.74.79:50010" + '
  '0.016*"blk_-7390710706320943439" + 0.016*"blk_-7385019746290967026" + '
  '0.016*"blk_-1160765867676592212" + 0.015*"10.251.31.5:50010" + '
  '0.015*"blk_-4130422002907235753" + 0.015*"blk_-372632911736985200"'),
 (3,
  '0.157*"block" + 0.151*"of" + 0.150*"from" + 0.150*"size" + 0.150*"Received" '
  '+ 0.146*"67108864" + 0.050*"1" + 0.001*"/10.251.31.5" + '
  '0.001*"/10.250.7.32" + 0.001*"/10.251.203.129"'),
 (4,
  '0.159*"block" + 0.145*"src:" + 0.145*"dest:" + 0.144*"Receiving" + '
  '0.005*"blk_4108032757710290196" + 0.005*"blk_-7326879742777330616" + '
  '0.005*"blk_4966276028193818154" + 0.005*"blk_-955678844525403431" + '
  '0.005*"blk_-4566764090560972175" + 0.005*"blk_7359588822015996941"'),
 (5,
  '0.014*"blk_-7199733435216834133" + 0.013*"blk_-3140754468249228022" + '
  '0.012*"blk_-8608758988186237206" + 0.012*"blk_-7374177486939901806" + '
  '0.012*"blk_7007568649123695530" + 0.011*"blk_252816358434654494" + '
  '0.011*"/10.251.126.5" + 0.011*"blk_4301761665496587132" + '
  '0.011*"/10.251.31.160" + 0.011*"/10.251.194.245"'),
 (6,
  '0.013*"/10.251.39.209" + 0.012*"/10.251.107.227" + '
  '0.011*"blk_-365681253977708548" + 0.011*"/10.251.203.179" + '
  '0.010*"/10.251.26.131" + 0.010*"/10.251.111.209" + 0.010*"/10.251.105.189" '
  '+ 0.009*"10.251.111.209:50010" + 0.009*"/10.251.43.21" + '
  '0.009*"/10.250.17.225"'),
 (7,
  '0.386*"for" + 0.157*"PacketResponder" + 0.156*"terminating" + 0.147*"block" '
  '+ 0.052*"0" + 0.052*"2" + 0.001*"/10.251.71.16" + 0.001*"/10.250.11.100" + '
  '0.001*"/10.251.197.226" + 0.001*"/10.250.14.196"'),
 (8,
  '0.320*"succeeded" + 0.320*"Verification" + 0.002*"blk_6420476111425645508" '
  '+ 0.002*"blk_4747238186020862016" + 0.002*"blk_-2382389751032389929" + '
  '0.002*"blk_7455154075126513420" + 0.002*"blk_4856031730010032819" + '
  '0.002*"blk_-3155415637543545820" + 0.002*"blk_638939784120214749" + '
  '0.002*"blk_-2947515393396507971"'),
 (9,
  '0.018*"blk_-9039789976760025618" + 0.018*"blk_8232653167951816215" + '
  '0.018*"blk_-3059453549958878582" + 0.017*"blk_9010689441969286891" + '
  '0.016*"blk_-1214394198778192417" + 0.015*"blk_-4358861442329520889" + '
  '0.015*"blk_-271233912558262172" + 0.014*"blk_5251082346485479848" + '
  '0.014*"blk_4472097162321285264" + 0.014*"blk_-4917509153767079107"')]

In [58]:
result_df_1 = pd.DataFrame([i[1].split(' + ') for i in _1st])

In [59]:
for i in result_df_1:
  result_df_1['{}_rate'.format(i)]=result_df_1[i].str[:5]
  result_df_1[i]= result_df_1[i].str[6:]

In [73]:
temp = []
for i in range(10):
  temp += ['{}_rate'.format(i), i]

In [75]:
result_df_1 = result_df_1[temp]

3차 topic

In [116]:
_3rd = [(0,
  '0.307*"block" + 0.015*"blk_-3059453549958878582" + 0.013*"/10.251.31.5" + '
  '0.012*"/10.251.126.5" + 0.011*"/10.250.7.32" + 0.011*"/10.251.203.129" + '
  '0.011*"/10.250.11.85" + 0.010*"/10.251.71.97" + '
  '0.010*"blk_-836233151529212612" + 0.009*"/10.251.39.192"'),
 (1,
  '0.234*"block" + 0.016*"blk_-2607603250149292767" + '
  '0.014*"blk_3126100339057783399" + 0.014*"10.251.74.79:50010" + '
  '0.014*"blk_-598955920782630733" + 0.013*"blk_4718466536212741328" + '
  '0.012*"/10.251.111.209" + 0.012*"10.251.111.209:50010" + '
  '0.011*"/10.251.111.37" + 0.011*"blk_7907797724760003565"'),
 (2,
  '0.200*"size" + 0.195*"67108864" + 0.101*"Received" + '
  '0.100*"NameSystem.addStoredBlock" + 0.100*"update" + 0.100*"blockMap" + '
  '0.091*"BLOCK*" + 0.001*"10.251.39.160:50010" + 0.001*"10.251.39.209:50010" '
  '+ 0.001*"10.251.107.227:50010"'),
 (3,
  '0.060*"BLOCK*" + 0.028*"NameSystem.allocateBlock" + '
  '0.004*"blk_8909107483987085802" + 0.004*"blk_9047244576407600764" + '
  '0.004*"blk_524408193881860019" + 0.004*"blk_4222802578485615752" + '
  '0.004*"blk_-2951262078924395064" + 0.004*"blk_-9081208173975932165" + '
  '0.004*"blk_8652779608730262978" + 0.004*"blk_-976339114197076853"'),
 (4,
  '0.318*"block" + 0.307*"PacketResponder" + 0.306*"terminate" + '
  '0.002*"/10.251.30.179" + 0.002*"10.251.90.64:50010" + '
  '0.001*"/10.250.11.100" + 0.001*"blk_-7722134978135978544" + '
  '0.001*"blk_5951221751432246380" + 0.001*"10.250.7.244:50010" + '
  '0.001*"blk_-1196019860597610288"'),
 (5,
  '0.020*"blk_-6742986629304373462" + 0.019*"blk_-5139777489517063680" + '
  '0.019*"blk_2907885683673342872" + 0.018*"blk_-3260736406913425394" + '
  '0.017*"10.251.106.10:50010" + 0.017*"blk_-861765373330722953" + '
  '0.016*"blk_-1402764052044494020" + 0.016*"blk_-3350625939026579183" + '
  '0.016*"blk_-7800853382581848688" + 0.013*"blk_4509878823587110787"'),
 (6,
  '0.265*"succeed" + 0.265*"Verification" + 0.097*"block" + '
  '0.002*"blk_-2382389751032389929" + 0.002*"blk_4747238186020862016" + '
  '0.002*"blk_6420476111425645508" + 0.002*"/10.251.39.209" + '
  '0.002*"blk_4856031730010032819" + 0.002*"blk_-3155415637543545820" + '
  '0.002*"/10.251.107.227"'),
 (7,
  '0.057*"blk_4241467193520768333" + 0.016*"blk_1037066694877136451" + '
  '0.016*"blk_-7325968241997525720" + 0.015*"blk_-7390710706320943439" + '
  '0.015*"blk_-1214394198778192417" + 0.015*"blk_-7374177486939901806" + '
  '0.014*"blk_-4358861442329520889" + 0.014*"blk_-372632911736985200" + '
  '0.013*"blk_-1322963786995945056" + 0.013*"blk_-7858897319223152664"'),
 (8,
  '0.192*"block" + 0.190*"dest" + 0.189*"Receiving" + '
  '0.007*"blk_4966276028193818154" + 0.007*"blk_4108032757710290196" + '
  '0.007*"blk_-7326879742777330616" + 0.006*"blk_-955678844525403431" + '
  '0.006*"blk_-4566764090560972175" + 0.006*"blk_8093107867996361765" + '
  '0.005*"blk_6157997286876178234"'),
 (9,
  '0.169*"block" + 0.017*"blk_4286887770593516283" + '
  '0.016*"blk_1207568959074804298" + 0.016*"blk_-8608758988186237206" + '
  '0.015*"10.251.31.5:50010" + 0.015*"blk_1200401807679484048" + '
  '0.014*"/10.251.194.245" + 0.013*"/10.251.105.189" + '
  '0.013*"blk_-6174155369373380263" + 0.012*"/10.251.126.227"')]

In [117]:
result_df_2 = pd.DataFrame([i[1].split(' + ') for i in _3rd])

In [118]:
for i in result_df_2:
  result_df_2['{}_rate'.format(i)]=result_df_2[i].str[:5]
  result_df_2[i]= result_df_2[i].str[6:]

In [119]:
temp = []
for i in range(10):
  temp += ['{}_rate'.format(i), i]

In [120]:
result_df_2 = result_df_2[temp]

결과 확인

In [90]:
result_df_1

,0_rate,0,1_rate,1,2_rate,2,3_rate,3,4_rate,4,5_rate,5,6_rate,6,7_rate,7,8_rate,8,9_rate,9
0,0.046,"""BLOCK*""",0.031,"""NameSystem.allocateBlock:""",0.012,"""blk_4241467193520768333""",0.004,"""blk_8909107483987085802""",0.004,"""blk_9047244576407600764""",0.004,"""blk_524408193881860019""",0.004,"""blk_4222802578485615752""",0.004,"""blk_-2951262078924395064""",0.004,"""blk_-9081208173975932165""",0.004,"""blk_8652779608730262978"""
1,0.101,"""to""",0.099,"""NameSystem.addStoredBlock:""",0.099,"""size""",0.099,"""is""",0.099,"""blockMap""",0.099,"""added""",0.099,"""updated:""",0.097,"""67108864""",0.095,"""BLOCK*""",0.001,"""10.251.39.160:50010"""
2,0.018,"""blk_-2607603250149292767""",0.017,"""blk_4286887770593516283""",0.017,"""blk_1037066694877136451""",0.016,"""10.251.74.79:50010""",0.016,"""blk_-7390710706320943439""",0.016,"""blk_-7385019746290967026""",0.016,"""blk_-1160765867676592212""",0.015,"""10.251.31.5:50010""",0.015,"""blk_-4130422002907235753""",0.015,"""blk_-372632911736985200"""
3,0.157,"""block""",0.151,"""of""",0.150,"""from""",0.150,"""size""",0.150,"""Received""",0.146,"""67108864""",0.050,"""1""",0.001,"""/10.251.31.5""",0.001,"""/10.250.7.32""",0.001,"""/10.251.203.129"""
4,0.159,"""block""",0.145,"""src:""",0.145,"""dest:""",0.144,"""Receiving""",0.005,"""blk_4108032757710290196""",0.005,"""blk_-7326879742777330616""",0.005,"""blk_4966276028193818154""",0.005,"""blk_-955678844525403431""",0.005,"""blk_-4566764090560972175""",0.005,"""blk_7359588822015996941"""
5,0.014,"""blk_-7199733435216834133""",0.013,"""blk_-3140754468249228022""",0.012,"""blk_-8608758988186237206""",0.012,"""blk_-7374177486939901806""",0.012,"""blk_7007568649123695530""",0.011,"""blk_252816358434654494""",0.011,"""/10.251.126.5""",0.011,"""blk_4301761665496587132""",0.011,"""/10.251.31.160""",0.011,"""/10.251.194.245"""
6,0.013,"""/10.251.39.209""",0.012,"""/10.251.107.227""",0.011,"""blk_-365681253977708548""",0.011,"""/10.251.203.179""",0.010,"""/10.251.26.131""",0.010,"""/10.251.111.209""",0.010,"""/10.251.105.189""",0.009,"""10.251.111.209:50010""",0.009,"""/10.251.43.21""",0.009,"""/10.250.17.225"""
7,0.386,"""for""",0.157,"""PacketResponder""",0.156,"""terminating""",0.147,"""block""",0.052,"""0""",0.052,"""2""",0.001,"""/10.251.71.16""",0.001,"""/10.250.11.100""",0.001,"""/10.251.197.226""",0.001,"""/10.250.14.196"""
8,0.320,"""succeeded""",0.320,"""Verification""",0.002,"""blk_6420476111425645508""",0.002,"""blk_4747238186020862016""",0.002,"""blk_-2382389751032389929""",0.002,"""blk_7455154075126513420""",0.002,"""blk_4856031730010032819""",0.002,"""blk_-3155415637543545820""",0.002,"""blk_638939784120214749""",0.002,"""blk_-2947515393396507971"""
9,0.018,"""blk_-9039789976760025618""",0.018,"""blk_8232653167951816215""",0.018,"""blk_-3059453549958878582""",0.017,"""blk_9010689441969286891""",0.016,"""blk_-1214394198778192417""",0.015,"""blk_-4358861442329520889""",0.015,"""blk_-271233912558262172""",0.014,"""blk_5251082346485479848""",0.014,"""blk_4472097162321285264""",0.014,"""blk_-4917509153767079107"""


In [121]:
result_df_2

,0_rate,0,1_rate,1,2_rate,2,3_rate,3,4_rate,4,5_rate,5,6_rate,6,7_rate,7,8_rate,8,9_rate,9
0,0.307,"""block""",0.015,"""blk_-3059453549958878582""",0.013,"""/10.251.31.5""",0.012,"""/10.251.126.5""",0.011,"""/10.250.7.32""",0.011,"""/10.251.203.129""",0.011,"""/10.250.11.85""",0.010,"""/10.251.71.97""",0.010,"""blk_-836233151529212612""",0.009,"""/10.251.39.192"""
1,0.234,"""block""",0.016,"""blk_-2607603250149292767""",0.014,"""blk_3126100339057783399""",0.014,"""10.251.74.79:50010""",0.014,"""blk_-598955920782630733""",0.013,"""blk_4718466536212741328""",0.012,"""/10.251.111.209""",0.012,"""10.251.111.209:50010""",0.011,"""/10.251.111.37""",0.011,"""blk_7907797724760003565"""
2,0.200,"""size""",0.195,"""67108864""",0.101,"""Received""",0.100,"""NameSystem.addStoredBlock""",0.100,"""update""",0.100,"""blockMap""",0.091,"""BLOCK*""",0.001,"""10.251.39.160:50010""",0.001,"""10.251.39.209:50010""",0.001,"""10.251.107.227:50010"""
3,0.060,"""BLOCK*""",0.028,"""NameSystem.allocateBlock""",0.004,"""blk_8909107483987085802""",0.004,"""blk_9047244576407600764""",0.004,"""blk_524408193881860019""",0.004,"""blk_4222802578485615752""",0.004,"""blk_-2951262078924395064""",0.004,"""blk_-9081208173975932165""",0.004,"""blk_8652779608730262978""",0.004,"""blk_-976339114197076853"""
4,0.318,"""block""",0.307,"""PacketResponder""",0.306,"""terminate""",0.002,"""/10.251.30.179""",0.002,"""10.251.90.64:50010""",0.001,"""/10.250.11.100""",0.001,"""blk_-7722134978135978544""",0.001,"""blk_5951221751432246380""",0.001,"""10.250.7.244:50010""",0.001,"""blk_-1196019860597610288"""
5,0.020,"""blk_-6742986629304373462""",0.019,"""blk_-5139777489517063680""",0.019,"""blk_2907885683673342872""",0.018,"""blk_-3260736406913425394""",0.017,"""10.251.106.10:50010""",0.017,"""blk_-861765373330722953""",0.016,"""blk_-1402764052044494020""",0.016,"""blk_-3350625939026579183""",0.016,"""blk_-7800853382581848688""",0.013,"""blk_4509878823587110787"""
6,0.265,"""succeed""",0.265,"""Verification""",0.097,"""block""",0.002,"""blk_-2382389751032389929""",0.002,"""blk_4747238186020862016""",0.002,"""blk_6420476111425645508""",0.002,"""/10.251.39.209""",0.002,"""blk_4856031730010032819""",0.002,"""blk_-3155415637543545820""",0.002,"""/10.251.107.227"""
7,0.057,"""blk_4241467193520768333""",0.016,"""blk_1037066694877136451""",0.016,"""blk_-7325968241997525720""",0.015,"""blk_-7390710706320943439""",0.015,"""blk_-1214394198778192417""",0.015,"""blk_-7374177486939901806""",0.014,"""blk_-4358861442329520889""",0.014,"""blk_-372632911736985200""",0.013,"""blk_-1322963786995945056""",0.013,"""blk_-7858897319223152664"""
8,0.192,"""block""",0.190,"""dest""",0.189,"""Receiving""",0.007,"""blk_4966276028193818154""",0.007,"""blk_4108032757710290196""",0.007,"""blk_-7326879742777330616""",0.006,"""blk_-955678844525403431""",0.006,"""blk_-4566764090560972175""",0.006,"""blk_8093107867996361765""",0.005,"""blk_6157997286876178234"""
9,0.169,"""block""",0.017,"""blk_4286887770593516283""",0.016,"""blk_1207568959074804298""",0.016,"""blk_-8608758988186237206""",0.015,"""10.251.31.5:50010""",0.015,"""blk_1200401807679484048""",0.014,"""/10.251.194.245""",0.013,"""/10.251.105.189""",0.013,"""blk_-6174155369373380263""",0.012,"""/10.251.126.227"""
